In [24]:
import utg
utg.data.WORDS_CACHES[r.WORD_TYPE.ADJECTIVE]

{(ADJECTIVE_FORM.FULL,
  GRADE.POSITIVE,
  ANIMALITY.ANIMATE,
  NUMBER.SINGULAR,
  CASE.NOMINATIVE,
  GENDER.MASCULINE): 0,
 (ADJECTIVE_FORM.FULL,
  GRADE.POSITIVE,
  ANIMALITY.ANIMATE,
  NUMBER.SINGULAR,
  CASE.NOMINATIVE,
  GENDER.NEUTER): 1,
 (ADJECTIVE_FORM.FULL,
  GRADE.POSITIVE,
  ANIMALITY.ANIMATE,
  NUMBER.SINGULAR,
  CASE.NOMINATIVE,
  GENDER.FEMININE): 2,
 (ADJECTIVE_FORM.FULL,
  GRADE.POSITIVE,
  ANIMALITY.ANIMATE,
  NUMBER.SINGULAR,
  CASE.GENITIVE,
  GENDER.MASCULINE): 3,
 (ADJECTIVE_FORM.FULL,
  GRADE.POSITIVE,
  ANIMALITY.ANIMATE,
  NUMBER.SINGULAR,
  CASE.GENITIVE,
  GENDER.NEUTER): 4,
 (ADJECTIVE_FORM.FULL,
  GRADE.POSITIVE,
  ANIMALITY.ANIMATE,
  NUMBER.SINGULAR,
  CASE.GENITIVE,
  GENDER.FEMININE): 5,
 (ADJECTIVE_FORM.FULL,
  GRADE.POSITIVE,
  ANIMALITY.ANIMATE,
  NUMBER.SINGULAR,
  CASE.DATIVE,
  GENDER.MASCULINE): 6,
 (ADJECTIVE_FORM.FULL,
  GRADE.POSITIVE,
  ANIMALITY.ANIMATE,
  NUMBER.SINGULAR,
  CASE.DATIVE,
  GENDER.NEUTER): 7,
 (ADJECTIVE_FORM.FULL,
  GRADE.PO

In [25]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

p = morph.parse('красная')[0]

lexems = p.lexeme



res = []

for i in range(len(lexems)):
    res.append(lexems[i][4][0][1])
    
print(res)

['красный', 'красного', 'красному', 'красного', 'красный', 'красным', 'красном', 'красная', 'красной', 'красной', 'красную', 'красной', 'красною', 'красной', 'красное', 'красного', 'красному', 'красное', 'красным', 'красном', 'красные', 'красных', 'красным', 'красных', 'красные', 'красными', 'красных', 'краснейший', 'краснейшего', 'краснейшему', 'краснейшего', 'краснейший', 'краснейшим', 'краснейшем', 'краснейшая', 'краснейшей', 'краснейшей', 'краснейшую', 'краснейшей', 'краснейшею', 'краснейшей', 'краснейшее', 'краснейшего', 'краснейшему', 'краснейшее', 'краснейшим', 'краснейшем', 'краснейшие', 'краснейших', 'краснейшим', 'краснейших', 'краснейшие', 'краснейшими', 'краснейших', 'красен', 'красна', 'красно', 'красны', 'краснее', 'красней', 'покраснее', 'покрасней']


In [10]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def generate_forms(word):
    p = morph.parse(word)[0]
    res = []
    lexems = p.lexeme
    if 'NOUN' in p.tag:
        for i in range(len(lexems)):
            res.append(lexems[i][4][0][1])
        #print(res)
        if 'femn' in p.tag:
            res1 = res[:5]
            res1.append(res[6])
            res1.extend(res[7:])
            res1.extend(res1[6:])
            return(res1)
        else:
            res.extend(res[6:])
            return(res)
    if 'VERB' in p.tag or 'INFN' in p.tag:
        for i in [0,1,3,2,4]:
            res.append(lexems[i][4][0][1])
        return(res)
    if 'ADJF' in p.tag:
        for i in range(18):
            res.append(lexems[i][4][0][1])
        return(res)
    
a = generate_forms('игрок')
a

['игрок',
 'игрока',
 'игроку',
 'игрока',
 'игроком',
 'игроке',
 'игроки',
 'игроков',
 'игрокам',
 'игроков',
 'игроками',
 'игроках',
 'игроки',
 'игроков',
 'игрокам',
 'игроков',
 'игроками',
 'игроках']

## Шаблон 1: игрок забивает гол противнику.

Задание: обработка имен и названий стран. 
Например: Акинфеев забил гол Франции.

In [11]:
from utg import relations as r
from utg import logic
from utg import data
from utg import dictionary
from utg import words
from utg import templates
from utg import constructors

# описываем существительное для словаря
noun_word = words.Word(type=r.WORD_TYPE.NOUN,
                        forms=generate_forms('гол'),
                        properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE)) # свойства: неодушевлённое, женский род

# описываем глагол для словаря
action_word = words.Word(type=r.WORD_TYPE.VERB,
                         # описываем только нужны нам формы слова (порядок важен и определён в utg.data.WORDS_CACHES[r.WORD_TYPE.VERB])
                         forms=generate_forms('забить') + [u''] * (len(data.WORDS_CACHES[r.WORD_TYPE.VERB]) - 5),
                         properties=words.Properties(r.ASPECT.PERFECTIVE, r.VOICE.DIRECT) )


# создаём словарь для использования в шаблонах
test_dictionary = dictionary.Dictionary(words=[noun_word, action_word])

# создаём шаблон
template = templates.Template()

# externals — внешние переменные, не обязаны быть в словаре
template.parse(u'[Subj] [забил|obj|прш] [obj|дт] [гол|вн].', externals=('hero', 'obj'))

# описываем внешние переменные
obj = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                 forms=generate_forms('противник'),
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.MASCULINE)))

subj = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                forms=generate_forms('игрок'),
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.MASCULINE)))


# осуществляем подстановку
result = template.substitute(externals={'subj': subj,
                                        'obj': obj},
                             dictionary=test_dictionary)

result 

'Игрок забил противнику гол.'

## Шаблон 2: игрок получает красную карточку.

Задание: обработка прилагательных

Например: красная

In [54]:
from utg import relations as r
from utg import logic
from utg import data
from utg import dictionary
from utg import words
from utg import templates
from utg import constructors

# описываем существительное для словаря
noun_word = words.Word(type=r.WORD_TYPE.NOUN,
                        forms=generate_forms('карточка'),
                        properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.FEMININE)) # свойства: неодушевлённое, женский род

# описываем глагол для словаря
action_word = words.Word(type=r.WORD_TYPE.VERB,
                         # описываем только нужны нам формы слова (порядок важен и определён в utg.data.WORDS_CACHES[r.WORD_TYPE.VERB])
                         forms=generate_forms('вручил') + [u''] * (len(data.WORDS_CACHES[r.WORD_TYPE.VERB]) - 5),
                         properties=words.Properties(r.ASPECT.PERFECTIVE, r.VOICE.DIRECT) )


# создаём словарь для использования в шаблонах
test_dictionary = dictionary.Dictionary(words=[noun_word, action_word])

# создаём шаблон
template = templates.Template()

# externals — внешние переменные, не обязаны быть в словаре
template.parse(u'[Obj] [вручил|obj] [subj|дт] [red|вн|жр] [card|вн].', externals=('subj', 'obj', 'card', 'red'))

# описываем внешние переменные
subj = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                 forms=generate_forms('игрок'),
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.MASCULINE)))

obj = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                forms=generate_forms('судья'),
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.MASCULINE)))
card = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                forms=generate_forms('карточка'),
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.FEMININE)))

red = words.WordForm(words.Word(type=r.WORD_TYPE.ADJECTIVE,
                                forms=generate_forms('красная')[8:] + [u''] * (8+len(data.WORDS_CACHES[r.WORD_TYPE.ADJECTIVE]) - len(generate_forms('красная'))) ,
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.FEMININE, r.CASE.NOMINATIVE)))

# осуществляем подстановку
result = template.substitute(externals={'subj': subj,
                                        'obj': obj,
                                       'card':card,
                                    'red':red},
                             dictionary=test_dictionary)

result 

'Судья вручил игроку красную карточку.'